In [ ]:
#Gives Team Roster
import time
from nba_api.stats.endpoints import CommonTeamRoster
from nba_api.stats.static import teams

# Function to fetch team roster with retry mechanism
def fetch_team_roster(team_id, season, retries=3):
    for attempt in range(retries):
        try:
            roster = CommonTeamRoster(team_id=team_id, season=season, timeout=60)
            return roster.get_data_frames()[0]
        except Exception as e:
            print(f"Attempt {attempt + 1}/{retries} failed: {e}")
            time.sleep(2)  # Wait before retrying
    return None

# Get all teams
all_teams = teams.get_teams()

# Get user input for team ID
team_id_input = input("Enter the team ID: ")

# Check if the team ID exists
if any(team['id'] == int(team_id_input) for team in all_teams):
    # Fetch the team roster
    roster_data = fetch_team_roster(team_id_input, season='2024-25')
    
    if roster_data is not None:
        print(f"\nRoster for team ID {team_id_input}:")
        for _, player in roster_data.iterrows():
            print(f"- {player['PLAYER']} (#{player['NUM']}, {player['POSITION']}, {player['HEIGHT']}, {player['WEIGHT']}, {player['AGE']} years old)")
    else:
        print("Failed to fetch the roster after multiple attempts.")
else:
    print("Invalid team ID.")


In [1]:
#Gives Team ID
from nba_api.stats.static import teams

# Get all teams
all_teams = teams.get_teams()

# Print team IDs along with their abbreviations and full names
print("List of NBA Teams and their IDs:")
for team in all_teams:
    print(f"Team ID: {team['id']}, Abbreviation: {team['abbreviation']}, Full Name: {team['full_name']}")


List of NBA Teams and their IDs:
Team ID: 1610612737, Abbreviation: ATL, Full Name: Atlanta Hawks
Team ID: 1610612738, Abbreviation: BOS, Full Name: Boston Celtics
Team ID: 1610612739, Abbreviation: CLE, Full Name: Cleveland Cavaliers
Team ID: 1610612740, Abbreviation: NOP, Full Name: New Orleans Pelicans
Team ID: 1610612741, Abbreviation: CHI, Full Name: Chicago Bulls
Team ID: 1610612742, Abbreviation: DAL, Full Name: Dallas Mavericks
Team ID: 1610612743, Abbreviation: DEN, Full Name: Denver Nuggets
Team ID: 1610612744, Abbreviation: GSW, Full Name: Golden State Warriors
Team ID: 1610612745, Abbreviation: HOU, Full Name: Houston Rockets
Team ID: 1610612746, Abbreviation: LAC, Full Name: Los Angeles Clippers
Team ID: 1610612747, Abbreviation: LAL, Full Name: Los Angeles Lakers
Team ID: 1610612748, Abbreviation: MIA, Full Name: Miami Heat
Team ID: 1610612749, Abbreviation: MIL, Full Name: Milwaukee Bucks
Team ID: 1610612750, Abbreviation: MIN, Full Name: Minnesota Timberwolves
Team ID: 

In [1]:
#Gives Player id From roster

from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster

def get_team_id(team_abb):
    # Find the team ID by abbreviation
    team = next((team for team in teams.get_teams() if team['abbreviation'] == team_abb), None)
    if team:
        return team['id']
    else:
        print(f"Team with abbreviation '{team_abb}' not found.")
        return None

def get_player_id(team_abb, player_name):
    team_id = get_team_id(team_abb)
    
    if not team_id:
        return None

    # Fetch the team's roster using CommonTeamRoster
    roster = commonteamroster.CommonTeamRoster(team_id=team_id)
    roster_data = roster.get_data_frames()[0]

    # Filter the roster for the player
    player_row = roster_data[roster_data['PLAYER'].str.contains(player_name, case=False)]
    
    if not player_row.empty:
        player_id = player_row.iloc[0]['PLAYER_ID']
        print(f"Player ID for {player_name} on {team_abb}: {player_id}")
        return player_id
    else:
        print(f"Player '{player_name}' not found on team '{team_abb}'.")
        return None

# Example usage:
team_abbreviation = input("Enter team abbreviation (e.g., LAL for Los Angeles Lakers): ")
player_name = input("Enter player's name: ")
get_player_id(team_abbreviation, player_name)


Player ID for LeBron James on LAL: 2544


2544

In [ ]:
#Rolling avg and avg vs every nba team
import pandas as pd
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import players

# Step 1: Get player game log stats for multiple seasons
def get_player_gamelog(player_id, seasons):
    all_games = []
    for season in seasons:
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]
        all_games.append(gamelog)
    return pd.concat(all_games, ignore_index=True)

# Step 2: Calculate rolling averages for last 5 and 10 games
def calculate_rolling_averages(player_data, stats_to_average):
    # Sort by game date to ensure the games are in chronological order
    player_data['GAME_DATE'] = pd.to_datetime(player_data['GAME_DATE'])
    player_data = player_data.sort_values(by='GAME_DATE', ascending=True)

    # Calculate rolling averages
    rolling_5 = player_data[stats_to_average].rolling(window=5).mean().iloc[-1]
    rolling_10 = player_data[stats_to_average].rolling(window=10).mean().iloc[-1]

    return rolling_5, rolling_10

# Step 3: Calculate averages against each team
def calculate_averages_vs_teams(player_data, stats_to_average):
    player_data['OPPONENT'] = player_data['MATCHUP'].apply(lambda x: x.split()[2])  # Extract opponent team abbreviation
    player_data['WL'] = player_data['WL'].map({'W': 1, 'L': 0})  # Convert 'W'/'L' to 1/0
    
    # Group by the opponent team and calculate average stats
    averages_vs_teams = player_data.groupby('OPPONENT').agg({
        'WL': 'mean',  # Win/Loss ratio against each team
        **{stat: 'mean' for stat in stats_to_average}
    }).reset_index()

    averages_vs_teams.rename(columns={'WL': 'Win_Loss_Ratio'}, inplace=True)
    return averages_vs_teams

# Main function
def main():
    # Get player and team data
    player_name = input("Enter a player's full name (e.g., LeBron James): ")
    nba_players = players.get_players()

    # Find player ID
    player_id = None
    for player in nba_players:
        if player['full_name'].lower() == player_name.lower():
            player_id = player['id']
            break

    if player_id is None:
        print("Player not found.")
        return

    # Seasons to retrieve data for (current season and last season)
    seasons = ['2024-25', '2023-24']

    # Get player game logs for these seasons
    player_data = get_player_gamelog(player_id, seasons)

    # Define the stats to calculate rolling averages for
    stats_to_average = ['PTS', 'REB', 'AST', 'TOV', 'FG_PCT', 'FT_PCT', 'FG3_PCT']  # Points, Rebounds, Assists, can add more stats if needed

    # Calculate rolling averages for the last 5 and 10 games
    rolling_5, rolling_10 = calculate_rolling_averages(player_data, stats_to_average)

    # Calculate averages vs each team
    averages_vs_teams = calculate_averages_vs_teams(player_data, stats_to_average)

    # Display the rolling averages
    print(f"\nRolling averages for the last 5 games:\n{rolling_5}")
    print(f"\nRolling averages for the last 10 games:\n{rolling_10}")

    # Display the averages against each team
    print(f"\nAverages vs each team for the current and last season:\n{averages_vs_teams}")

if __name__ == "__main__":
    main()


In [ ]:
#NUmber of times player got more then their average last season
import pandas as pd
from nba_api.stats.endpoints import commonteamroster, playergamelog
from nba_api.stats.static import teams

# Step 1: Get the team roster
def get_team_roster(team_id):
    try:
        team_roster = commonteamroster.CommonTeamRoster(team_id=team_id).get_data_frames()
        if not team_roster:
            print("No roster data found.")
            return None
        roster_df = team_roster[0]
        print(f"Roster DataFrame Columns: {roster_df.columns.tolist()}")  # Print columns for debugging
        return roster_df
    except Exception as e:
        print(f"Error retrieving team roster: {e}")
        return None

# Step 2: Get player game logs for the last season
def get_player_gamelog(player_id, season):
    try:
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()
        if not gamelog:
            print("No game log data found.")
            return None
        return gamelog[0]
    except Exception as e:
        print(f"Error retrieving game logs for player {player_id}: {e}")
        return None

# Step 3: Count how many times each player exceeded their average stats
def count_exceeding_average(team_id, season):
    # Get team roster
    roster = get_team_roster(team_id)
    if roster is None:
        return

    for index, player in roster.iterrows():
        player_id = player['PLAYER_ID']
        player_name = player['PLAYER'] if 'PLAYER' in player else 'Unknown Player'
        
        # Get player game logs
        player_data = get_player_gamelog(player_id, season)
        if player_data is None or player_data.empty:
            continue
        
        # Calculate average stats
        avg_points = player_data['PTS'].mean()
        avg_rebounds = player_data['REB'].mean()
        avg_assists = player_data['AST'].mean()
        avg_turnovers = player_data['TOV'].mean()

        # Check for NaN averages
        if pd.isna(avg_points):
            print(f"{player_name} has no valid average points for the {season} season.")
            continue
        
        # Count instances where stats exceeded averages
        exceeding_points_count = (player_data['PTS'] > avg_points).sum()
        exceeding_rebounds_count = (player_data['REB'] > avg_rebounds).sum()
        exceeding_assists_count = (player_data['AST'] > avg_assists).sum()
        exceeding_turnovers_count = (player_data['TOV'] > avg_turnovers).sum()

        # Print results
        print(f"{player_name} exceeded their average points ({avg_points:.2f}) {exceeding_points_count} times in the {season} season.")
        print(f"{player_name} exceeded their average rebounds ({avg_rebounds:.2f}) {exceeding_rebounds_count} times in the {season} season.")
        print(f"{player_name} exceeded their average assists ({avg_assists:.2f}) {exceeding_assists_count} times in the {season} season.")
        print(f"{player_name} exceeded their average turnovers ({avg_turnovers:.2f}) {exceeding_turnovers_count} times in the {season} season.")
        print()  # Blank line for readability

# Example usage
team_id = input("Enter the team ID (e.g., 1610612747 for the Lakers): ")
season = '2023-24'  # Last season
count_exceeding_average(team_id, season)


In [ ]:
#NUmber of times player got more then their average this season
import pandas as pd
from nba_api.stats.endpoints import commonteamroster, playergamelog
from nba_api.stats.static import teams

# Step 1: Get the team roster
def get_team_roster(team_id):
    try:
        team_roster = commonteamroster.CommonTeamRoster(team_id=team_id).get_data_frames()
        if not team_roster:
            print("No roster data found.")
            return None
        roster_df = team_roster[0]
        print(f"Roster DataFrame Columns: {roster_df.columns.tolist()}")  # Print columns for debugging
        return roster_df
    except Exception as e:
        print(f"Error retrieving team roster: {e}")
        return None

# Step 2: Get player game logs for the last season
def get_player_gamelog(player_id, season):
    try:
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()
        if not gamelog:
            print("No game log data found.")
            return None
        return gamelog[0]
    except Exception as e:
        print(f"Error retrieving game logs for player {player_id}: {e}")
        return None

# Step 3: Count how many times each player exceeded their average stats
def count_exceeding_average(team_id, season):
    # Get team roster
    roster = get_team_roster(team_id)
    if roster is None:
        return

    for index, player in roster.iterrows():
        player_id = player['PLAYER_ID']
        player_name = player['PLAYER'] if 'PLAYER' in player else 'Unknown Player'
        
        # Get player game logs
        player_data = get_player_gamelog(player_id, season)
        if player_data is None or player_data.empty:
            continue
        
        # Calculate average stats
        avg_points = player_data['PTS'].mean()
        avg_rebounds = player_data['REB'].mean()
        avg_assists = player_data['AST'].mean()
        avg_turnovers = player_data['TOV'].mean()

        # Check for NaN averages
        if pd.isna(avg_points):
            print(f"{player_name} has no valid average points for the {season} season.")
            continue
        
        # Count instances where stats exceeded averages
        exceeding_points_count = (player_data['PTS'] > avg_points).sum()
        exceeding_rebounds_count = (player_data['REB'] > avg_rebounds).sum()
        exceeding_assists_count = (player_data['AST'] > avg_assists).sum()
        exceeding_turnovers_count = (player_data['TOV'] > avg_turnovers).sum()

        # Print results
        print(f"{player_name} exceeded their average points ({avg_points:.2f}) {exceeding_points_count} times in the {season} season.")
        print(f"{player_name} exceeded their average rebounds ({avg_rebounds:.2f}) {exceeding_rebounds_count} times in the {season} season.")
        print(f"{player_name} exceeded their average assists ({avg_assists:.2f}) {exceeding_assists_count} times in the {season} season.")
        print(f"{player_name} exceeded their average turnovers ({avg_turnovers:.2f}) {exceeding_turnovers_count} times in the {season} season.")
        print()  # Blank line for readability

# Example usage
team_id = input("Enter the team ID (e.g., 1610612747 for the Lakers): ")
season = '2024-25'  # Last season
count_exceeding_average(team_id, season)
